In [ ]:
# Analysis of COVID-19 specific DMPs in comparison with not hospitalized and healthy controls

In [ ]:
import pandas as pd
import plotly.express as px
from IPython.display import display

import scipy.stats as sts
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt

from src.figures import boxplot
from src.col_palette import pal

In [ ]:
# sample name mapper
names = {
    "Home": "A_not Hospitalized",
    "ICU": "Hospitalized",
    "non-ICU": "Hospitalized",
}

In [ ]:
samplesheeet = pd.read_csv("../data/raw/SampleSheet.csv", index_col=0).dropna()
samplesheeet.Sex = samplesheeet.Sex.replace({"F": 1, "M": 0})
samplesheeet["H_Status"] = samplesheeet.ICU.replace(names)

In [ ]:
samplesheeet = samplesheeet[~samplesheeet["Status"].str.contains("Other")]
samplesheeet = samplesheeet[["H_Status", "Sex", "Age"]]
samplesheeet

In [ ]:
samplesheeet.H_Status.value_counts()

In [ ]:
dmps = pd.read_csv("../Files/COVSpecificDMPs.csv", index_col=0)
dmps.UCSC_RefGene_Name = dmps.UCSC_RefGene_Name.fillna("")
dmps.UCSC_RefGene_Name = dmps.UCSC_RefGene_Name.str.split(";").map(
    lambda x: " ".join(set(x))
)
dmps["annotation"] = dmps.index + " " + dmps.UCSC_RefGene_Name

In [ ]:
len(dmps.index)

In [ ]:
mynorm = pd.read_parquet(
    "../data/processed/CorrectedMyNorms/mynorm.parquet", columns=samplesheeet.index
).T

mynorm = mynorm[dmps.index]

In [ ]:
mynorm = pd.concat((mynorm, samplesheeet), axis=1)
mynorm = mynorm.sort_values("H_Status")
mynorm

In [ ]:
records = []
diff = []

for cnt, cpg in enumerate(dmps.index):

    temp_df = mynorm[[cpg, "Sex", "Age"]]
    temp_df["intercept"] = 1

    model = sm.MNLogit(
        exog=temp_df,
        endog=mynorm["H_Status"],
    )

    model = model.fit()

    print(model.summary())

    pvals = model.pvalues.loc[cpg]
    pvals.index = ["Healthy controls", "Hospitalized"]

    pvals = pvals.to_frame().T
    records.append(pvals)

    hb_mean = mynorm[mynorm["H_Status"] == "Healthy controls"][cpg].mean()
    not_hosp_mean = mynorm[mynorm["H_Status"] == "A_not Hospitalized"][cpg].mean()
    hosp_mean = mynorm[mynorm["H_Status"] == "Hospitalized"][cpg].mean()

    diff.append(
        {
            "CpG": cpg,
            "Healthy controls - not Hospitalized": hb_mean - not_hosp_mean,
            "Healthy controls - Hospitalized": hb_mean - hosp_mean,
            "not Hospitalized - Hospitalized": not_hosp_mean - hosp_mean,
        }
    )


records = pd.concat(records)
_, records["FDR Healthy controls"], _, _ = multipletests(
    records["Healthy controls"], method="fdr_bh"
)

_, records["FDR Hospitalized"], _, _ = multipletests(
    records["Hospitalized"], method="fdr_bh"
)

diff = pd.DataFrame(diff)
diff = diff.set_index("CpG")

results = pd.concat((diff, records), axis=1)

dmps_index = results[
    (results["FDR Healthy controls"] <= 0.05)
    & (results["FDR Hospitalized"] <= 0.05)
    & (results["Healthy controls - not Hospitalized"].abs() > 0.05)
    & (results["Healthy controls - Hospitalized"].abs() > 0.05)
    & (results["not Hospitalized - Hospitalized"].abs() > 0.05)
].index

results.loc[dmps_index, "DMP"] = True
results["DMP"] = results["DMP"].fillna(False)

In [ ]:
results.to_csv("../Files/StatsHospitalization.csv")
results

In [ ]:
mapper = dict(zip(dmps.index, dmps.annotation))

In [ ]:
try:
    mynorm = mynorm.drop(["Sex", "Age"], axis=1)
except:
    pass

melted = mynorm.melt("H_Status", var_name="CpG", value_name="β-value")
melted.H_Status = melted.H_Status.replace({"A_not Hospitalized": "not Hospitalized"})

In [ ]:
melted.H_Status.unique()

In [ ]:
boxplot(
    melted,
    category_orders={
        "H_Status": ["Healthy controls", "not Hospitalized", "Hospitalized"]
    },
    y="β-value",
    facet_col="CpG",
    color_column="H_Status",
    facet_col_wrap=6,
    width=1800,
    height=800,
    facet_font_size=18,
    path="../Plots/Hosp_notHosp.png",
)